## 3日目の演習
#### 量子回路の深さを気にしなければ、どのような量子状態も任意の精度で表現することが可能です。
#### しかし、深い量子回路を使えば、現状の量子デバイスで計算するとノイズがどんどん重なり、目的の量子状態とはかけ離れたものが得られてしまいます。
#### 現在の量子計算では、量子状態の表現力と量子状態へのノイズの影響はトレードオフ
#### そこで、目的の量子状態を近似的に表現することが可能な比較的浅い量子回路（Ansatz）の構成法が研究されています。

### ここでは様々なAnsatzでどのような量子状態が構成できるのか確認する

### A. 1量子ビットを表現する量子回路

## パラメータ付き量子回路

### 1. パラメータ付き量子回路を作る

In [ ]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket import Circuit, OpType
from pytket.circuit import fresh_symbol
from sympy import Symbol, symbols

circ1 = Circuit()
circ1.add_q_register('ansatz', 1)
a = symbols("a")
#a = fresh_symbol("a")
circ1.add_gate(OpType.Rx, [a], [circ1.qubits[0]])
circ1.measure_all()
render_circuit_jupyter(circ1)

circ2 = Circuit()
circ2.add_q_register('ansatz', 1)
a11, a12, a13 = symbols("a11, a12, a13")
#a11 = fresh_symbol("a11")
#a12 = fresh_symbol("a12")
#a13 = fresh_symbol("a13")
circ2.add_gate(OpType.TK1, [a11,a12,a13], [circ2.qubits[0]])
circ2.measure_all()
render_circuit_jupyter(circ2)

TK1はオイラー角
- `TK1(a11,a12,a13)=Rz(a11)Rx(a12)Rz(a13)`

### 2. パラメータ付き量子回路のパラメータに値を代入する

In [ ]:
s_map1 = {a: 0.25}
circ1.symbol_substitution(s_map1)
render_circuit_jupyter(circ1)

s_map2 = {a11: 0, a12: 1/2, a13: 0}
circ2.symbol_substitution(s_map2)
render_circuit_jupyter(circ2)

### 3. 量子回路を測定してみる

In [ ]:
# IMBのシミュレータの準備
from pytket.extensions.qiskit import AerBackend
backend = AerBackend()

In [ ]:
# 量子回路をIBMのシミュレータのゲートセットで書き直す
compiled_circ1 = backend.get_compiled_circuit(circ1)
render_circuit_jupyter(compiled_circ1)
compiled_circ2 = backend.get_compiled_circuit(circ2)
render_circuit_jupyter(compiled_circ2)

In [ ]:
# 1000 shot測定をして、結果を集計
from qiskit.visualization import plot_histogram
shot = 1000
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result_counts1 = backend.get_result(handle1).get_counts()
#result_sum=0
for i in result_counts1.keys():
    print(i, result_counts1[i]/shot)
#    result_sum += result_counts1[i]
#print(result_sum)
plot_histogram(result_counts1)

In [ ]:
# 1000 shot測定をして、結果を集計
from qiskit.visualization import plot_histogram
shot = 1000
handle2 = backend.process_circuit(compiled_circ2, n_shots=shot)
result_counts2 = backend.get_result(handle2).get_counts()
plot_histogram(result_counts2)

## 目的の量子状態 ( n量子ビット ) $\left|\psi\rangle\right.$ を表現する量子回路について考えてみる

### 1. おもちゃのモデル：与えられた量子状態 $\left|\psi\rangle\right.$を表現する量子回路の構成を考えてみる。


問：与えられた量子状態 $\left|\psi\rangle\right.$に対して、量子回路$U(\theta)$で
$$
U(\theta) \left|\, 0^{\otimes n}\rangle\right. 
$$
が$\left|\psi\rangle\right.$となる or $\left|\psi\rangle\right.$とほぼ等しくなる量子回路$U(\theta)$が見つけられるか。

課題：
- 量子回路$U(\theta)$をどのように構成するか？（浅ければ浅いほど良い）
- どのようにして $\theta$を算出するのか？

#### 推定したい量子ビット
$$
|\psi\rangle=|-\rangle=\frac{1}{\sqrt{2}}|0\rangle-\frac{1}{\sqrt{2}}|1\rangle
$$

#### 推定したい量子ビットを準備 


In [ ]:
import numpy as np
from numpy import pi

In [ ]:
# 推定したい量子状態
psi=(np.cos(pi/4),np.exp(pi*1j)*np.sin(pi/4))
psi

In [ ]:
#psiの直交ベクトル
psip=(np.sin(pi/4),-np.exp(pi*1j)*np.cos(pi/4))
psip

In [ ]:
from pytket.circuit import Circuit, Unitary1qBox
from pytket.circuit.display import render_circuit_jupyter
import numpy as np

psi_u = np.asarray([[psi[0], psip[0]],
                 [psi[1], psip[1]]])
#psi = np.asarray([[0.5403023058681398, 0.8414709848078965],
#                 [(-0.35017548837401463+0.7651474012342926j), (0.2248450953661529-0.49129549643388193j)]])
print(psi_u)

dagger_psi_u = np.conjugate(psi_u.T)
#dagger_psi_u = asarray([[psi[0], np.conjugate(psi[1])],
#                 [psip[0], np.conjugate(psip[1])]])
#dagger_psi = np.asarray([[0.5403023058681398, (-0.35017548837401463-0.7651474012342926j)],
#                 [0.8414709848078965, (0.2248450953661529+0.49129549643388193j)]])
print(dagger_psi_u)
#print(np.dot(dagger_psi_u,psi_u))

In [ ]:
# braベクトル<psi^dagger|を構成
circ_opt = Circuit()
circ_opt.add_q_register("q",1)
#u1box_psi = Unitary1qBox(psi_u)
u1box_psid = Unitary1qBox(dagger_psi_u)
circ_opt.add_unitary1qbox(u1box_psid, circ_opt.qubits[0])
circ_opt.measure_all()
render_circuit_jupyter(circ_opt)

## $U(\theta)$を定め、パラメータ $\theta$ を算出する

In [ ]:
from pytket.circuit import Circuit, OpType, Unitary1qBox
from pytket.circuit.display import render_circuit_jupyter
from qiskit.visualization import plot_histogram
from pytket.extensions.qiskit import AerBackend
from sympy import symbols
import numpy as np
from numpy import pi

初期値$\theta_0$を選択肢し、最小2乗法を利用する。

最小2乗法：誤差の2乗和を最小にするようパラメータを最適化する。

In [ ]:
psi=(np.cos(pi/4),np.exp(pi*1j)*np.sin(pi/4))
psip=(np.sin(pi/4),-np.exp(pi*1j)*np.cos(pi/4))
psi_u = np.asarray([[psi[0], psip[0]],
                 [psi[1], psip[1]]])
dagger_psi_u = np.conjugate(psi_u.T)
u1box_psid = Unitary1qBox(dagger_psi_u)
backend = AerBackend()

誤差（目的関数、損失関数）を定義

In [ ]:
def var(res):
    v=0
    n=0
    for i in res.values():
        v += ((1 - n) - i/shot)**2
        n += 1 
    return v

#### 1. Ansatz $U(\theta)=Rx(\theta)$とする

まずは、誤差varを算出

#### 量子回路を作成

In [ ]:
# 量子回路作成
t = symbols("t")
circx = Circuit(1)
circx.add_gate(OpType.Rx,[t],[0])
circx.add_unitary1qbox(u1box_psid, 0)
circx.measure_all()
render_circuit_jupyter(circx)

#### 初期値 $t=0$ を代入

In [ ]:
t0 = 0
t_map = {t: t0}
circ_opt = circx.copy()
circ_opt.symbol_substitution(t_map)
render_circuit_jupyter(circ_opt)

#### 測定

In [ ]:
shot = 1000000
compiled_circ = backend.get_compiled_circuit(circ_opt)
handle = backend.process_circuit(compiled_circ, n_shots=shot)
result = backend.get_result(handle).get_counts()
for i in result.keys():
    print(i, result[i]/shot)
plot_histogram(result)

In [ ]:
var1 = var(result)
print('誤差：', var1)

#### Newton法を使って、varの値を漸近的に0近づける
epsilon=0.0001

In [ ]:
epsilon=0.0001

In [ ]:
t_map1 = {t: t0-epsilon}
circ_opt1 = circx.copy()
circ_opt1.symbol_substitution(t_map1)
render_circuit_jupyter(circ_opt1)

In [ ]:
compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result1 = backend.get_result(handle1).get_counts()
for i in result1.keys():
    print(i, result1[i]/shot)
plot_histogram(result1)

In [ ]:
var2 = var(result1)
var2

varのt=0での勾配算出

In [ ]:
(var1-var2)/epsilon

更新率 alpha=0.001

In [ ]:
alpha=0.001

パラメータを更新し、繰り返す

In [ ]:
new_t = t0-alpha*(var1-var2)/epsilon

In [ ]:
t_map = {t: new_t}
circ_opt = circx.copy()
circ_opt.symbol_substitution(t_map)
render_circuit_jupyter(circ_opt)

In [ ]:
compiled_circ = backend.get_compiled_circuit(circ_opt)
handle = backend.process_circuit(compiled_circ, n_shots=shot)
result = backend.get_result(handle).get_counts()
for i in result.keys():
    print(i, result[i]/shot)
var1 = var(result)
print(var1)

In [ ]:
t_map1 = {t: new_t-epsilon}
circ_opt1 = circx.copy()
circ_opt1.symbol_substitution(t_map1)
compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result1 = backend.get_result(handle1).get_counts()
for i in result1.keys():
    print(i, result1[i]/shot)
var2 = var(result1)
print(var2)

In [ ]:
(var1-var2)/epsilon

In [ ]:
new_t = new_t-alpha*(var1-var2)/epsilon
t_map = {t: new_t}
circ_opt = circx.copy()
circ_opt.symbol_substitution(t_map)
compiled_circ = backend.get_compiled_circuit(circ_opt)
handle = backend.process_circuit(compiled_circ, n_shots=shot)
result = backend.get_result(handle).get_counts()
for i in result.keys():
    print(i, result[i]/shot)
var1 = var(result)
print(var1)

In [ ]:
t_map1 = {t: new_t-epsilon}
circ_opt1 = circx.copy()
circ_opt1.symbol_substitution(t_map1)
compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result1 = backend.get_result(handle1).get_counts()
for i in result1.keys():
    print(i, result1[i]/shot)
var2 = var(result1) 
print(var2)
print('勾配：',(var1-var2)/epsilon)

In [ ]:
new_t = new_t-alpha*(var1-var2)/epsilon
t_map = {t: new_t}
circ_opt = circx.copy()
circ_opt.symbol_substitution(t_map)
compiled_circ = backend.get_compiled_circuit(circ_opt)
handle = backend.process_circuit(compiled_circ, n_shots=shot)
result = backend.get_result(handle).get_counts()
for i in result.keys():
    print(i, result[i]/shot)
var1 = var(result)
print('誤差：',var1)
t_map1 = {t: new_t-epsilon}
circ_opt1 = circx.copy()
circ_opt1.symbol_substitution(t_map1)
compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result1 = backend.get_result(handle1).get_counts()
for i in result1.keys():
    print(i, result1[i]/shot)
var2 = var(result1)
#print(var2)
print('勾配：',(var1-var2)/epsilon)

In [ ]:
time=0
while (var1 > 0.01 and time < 10):
    new_t = new_t-alpha*(var1-var2)/epsilon
    t_map = {t: new_t}
    circ_opt = circx.copy()
    circ_opt.symbol_substitution(t_map)
    compiled_circ = backend.get_compiled_circuit(circ_opt)
    handle = backend.process_circuit(compiled_circ, n_shots=shot)
    result = backend.get_result(handle).get_counts()
    for i in result.keys():
        print(i, result[i]/shot)
    var1 = var(result)
    print('誤差：',var1)
    t_map1 = {t: new_t-epsilon}
    circ_opt1 = circx.copy()
    circ_opt1.symbol_substitution(t_map1)
    compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
    handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
    result1 = backend.get_result(handle1).get_counts()
    for i in result1.keys():
        print(i, result1[i]/shot)
    var2 = var(result1)
    #print(var2)
    print('勾配：',(var1-var2)/epsilon)
    time += 1

#### 2. Ansatz $U(\theta)=Ry(\theta)$とする

In [ ]:
# 量子回路作成
t = symbols("t")
circy = Circuit(1)
circy.add_gate(OpType.Ry,[t],[0])
circy.add_unitary1qbox(u1box_psid, 0)
circy.measure_all()
render_circuit_jupyter(circy)

In [ ]:
t0 = 0
t_map = {t: t0}
circ_opt = circy.copy()
circ_opt.symbol_substitution(t_map)
render_circuit_jupyter(circ_opt)

In [ ]:
shot = 10000
compiled_circ = backend.get_compiled_circuit(circ_opt)
handle = backend.process_circuit(compiled_circ, n_shots=shot)
result = backend.get_result(handle).get_counts()
for i in result.keys():
    print(i, result[i]/shot)
var1 = var(result)
print('誤差：',var1)

In [ ]:
epsilon=0.001

In [ ]:
t_map1 = {t: t0-epsilon}
circ_opt1 = circy.copy()
circ_opt1.symbol_substitution(t_map1)
render_circuit_jupyter(circ_opt1)

In [ ]:
compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result1 = backend.get_result(handle1).get_counts()
for i in result1.keys():
    print(i, result1[i]/shot)
var2 = var(result1)
print(var2)
print('勾配：',(var1-var2)/epsilon)

In [ ]:
alpha=0.05

In [ ]:
new_t = t0-alpha*(var1-var2)/epsilon
t_map = {t: new_t}
circ_opt = circy.copy()
circ_opt.symbol_substitution(t_map)
compiled_circ = backend.get_compiled_circuit(circ_opt)
handle = backend.process_circuit(compiled_circ, n_shots=shot)
result = backend.get_result(handle).get_counts()
for i in result.keys():
    print(i, result[i]/shot)
var1 = var(result)
print('誤差：',var1)
t_map1 = {t: new_t-epsilon}
circ_opt1 = circy.copy()
circ_opt1.symbol_substitution(t_map1)
compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result1 = backend.get_result(handle1).get_counts()
for i in result1.keys():
    print(i, result1[i]/shot)
var2 = var(result1) 
print(var2)
print('勾配：',(var1-var2)/epsilon)

In [ ]:
alpha=0.05

In [ ]:
new_t = new_t-alpha*(var1-var2)/epsilon
t_map = {t: new_t}
circ_opt = circy.copy()
circ_opt.symbol_substitution(t_map)
compiled_circ = backend.get_compiled_circuit(circ_opt)
handle = backend.process_circuit(compiled_circ, n_shots=shot)
result = backend.get_result(handle).get_counts()
for i in result.keys():
    print(i, result[i]/shot)
var1 = var(result)
print('誤差：',var1)
t_map1 = {t: new_t-epsilon}
circ_opt1 = circy.copy()
circ_opt1.symbol_substitution(t_map1)
compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
result1 = backend.get_result(handle1).get_counts()
for i in result1.keys():
    print(i, result1[i]/shot)
var2 = var(result1)
print(var2)
print('勾配：',(var1-var2)/epsilon)

In [ ]:
while var1 > 0.00001:
    new_t = new_t-alpha*(var1-var2)/epsilon
    t_map = {t: new_t}
    circ_opt = circy.copy()
    circ_opt.symbol_substitution(t_map)
    compiled_circ = backend.get_compiled_circuit(circ_opt)
    handle = backend.process_circuit(compiled_circ, n_shots=shot)
    result = backend.get_result(handle).get_counts()
    for i in result.keys():
        print(i, result[i]/shot)
    var1 = var(result)
    print('誤差：',var1)
    t_map1 = {t: new_t-epsilon}
    circ_opt1 = circy.copy()
    circ_opt1.symbol_substitution(t_map1)
    compiled_circ1 = backend.get_compiled_circuit(circ_opt1)
    handle1 = backend.process_circuit(compiled_circ1, n_shots=shot)
    result1 = backend.get_result(handle1).get_counts()
    for i in result1.keys():
        print(i, result1[i]/shot)
    var2 = var(result1)
    print(var2)
    print('勾配：',(var1-var2)/epsilon)
    print(new_t)

In [ ]:
psi

In [ ]:
circres=Circuit(1).add_gate(OpType.Ry,[new_t],[0])
print(circres.get_statevector())

#### 3. Ansatz $U(\theta)=Rx(\theta_1)Ry(\theta_2)$を使って、下記の $|\psi\rangle$ を表現する量子回路を作る

In [ ]:
from pytket.circuit import Circuit, OpType, Unitary1qBox
from pytket.circuit.display import render_circuit_jupyter
from qiskit.visualization import plot_histogram
from pytket.extensions.qiskit import AerBackend
from sympy import symbols
import numpy as np
from numpy import pi
from day3 import *

In [ ]:
psi

$U(\theta)=Rx(\theta_1)Ry(\theta_2)$とした量子回路を作成してください

$\theta_1=1, \theta_2=1$を代入した新しい量子回路を作成してください

IBMのシミュレータで10000ショットを実行してください。誤差を確認

In [ ]:
backend = AerBackend()
shot = 10000

In [ ]:
compiled_circ = #埋めてください
handle = #埋めてください
result = #埋めてください
for i in result.keys():
    print(i, result[i]/shot)
var1 = var(result)
print('誤差:',var1)

$\epsilon=0.001$とし、$\theta_1=1-\epsilon$,$\theta_2=1$を代入した新しい回路を作成し、10000ショットを実行。

$\theta_1=1$,$\theta_2=1$ の結果と合わせて, 誤差の勾配を計算

In [ ]:
t_map1 = {: ,: }　#theta1に1-epsilon,theta2に1を代入した新しい回路を作成
circ_opt1 = circ.copy()
circ_opt1.symbol_substitution(t_map1)
render_circuit_jupyter(circ_opt1)

In [ ]:
#勾配を計算
compiled_circ1 = #埋めてください
handle1 = #埋めてください
result1 = #埋めてください
for i in result1.keys():
    print(i, result1[i]/shot)
var01 = var(result1)
print(var01)
print((var1-var01)/epsilon)#誤差の勾配

更新率0.05としてパラメータ$\theta_1$を更新してください

In [ ]:
#更新率
alpha = 0.05
#パラメータの更新
      =

パラメータ$\theta_1$を更新した新しい量子回路を作成してください

In [ ]:
t_map = {:, :}
circ_opt = circ.copy()
circ_opt.symbol_substitution(t_map)
render_circuit_jupyter(circ_opt)

同様にパラメータ$\theta_2$を更新してください

パラメータ$\theta_1$,$\theta_2$を交互に更新し、誤差が $0.0001$以下になるまで更新してください。

得られた量子回路を考察してください。

IBMの実機で確認してください。